In [2]:
from pdastro import pdastroclass,pdastrostatsclass,makepath4file,unique,AnotB,AorB,AandB,rmfile
import pandas as pd
import numpy as np
import glob,re,os,sys

PRD_dir = '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701'

# This is the file with the new v2v3 parameters
newv2v3filename = f'{PRD_dir}/jwst_fpa.01074.results.v2v3info.mean.txt'
sw_reffilter = 'f210m'
lw_reffilter = 'f335m'

# filepattern for the distortion coefficient text files
distcoeff_filepattern = f'{PRD_dir}/*distcoeff.txt'
# make a dictionary that points from detector name to distortion file
distcoeff_files = glob.glob(distcoeff_filepattern)
detector2distcoeff = {}
for distcoeff_file in distcoeff_files:
    m = re.search('(^[a-zA-Z0-9]+)',os.path.basename(distcoeff_file))
    if m is None: 
        raise RuntimeError(f'could not parse {distcoeff_file}')
    detector = m.groups()[0]
    detector2distcoeff[detector]=distcoeff_file
print(detector2distcoeff)

# Load the siaf table
siaftable = pdastroclass()
### Make sure this is the file from the newest prd! 
### https://github.com/spacetelescope/pysiaf/blob/master/pysiaf/prd_data/JWST/PRDOPSSOC-051/SIAFXML/SIAFXML/NIRCam_SIAF.xml
### NIRCam_SIAF.prd51.xml is from the ^ link
siafxml = '/Users/arest/nircam/jwst_distortions_tools/NIRCam_SIAF.prd51.xml'


siaftable.t = pd.read_xml(siafxml)
print(siaftable.t.columns)
print('Example: nrca3:')
ixs_nrca3 = siaftable.ix_equal('AperName','NRCA3_FULL')
siaftable.write(columns=['AperName','V2Ref','V3Ref','V3IdlYAngle','V3SciXAngle','V3SciYAngle'],indices=ixs_nrca3)

{'nrca3': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrca3_full_f150w_clear.distcoeff.txt', 'nrcb4': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrcb4_full_f150w_clear.distcoeff.txt', 'nrcb2': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrcb2_full_f150w_clear.distcoeff.txt', 'nrca4': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrca4_full_f150w_clear.distcoeff.txt', 'nrca2': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrca2_full_f150w_clear.distcoeff.txt', 'nrcb3': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrcb3_full_f150w_clear.distcoeff.txt', 'nrca1': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrca1_full_f150w_clear.distcoeff.txt', 'nrca5': '/Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/nrca5_full_f356w_clear.distcoeff.txt', 'nrcb5': '/Users/arest/nircam/jwst_distortions_

0

In [26]:
newv2v3table = pdastroclass()
newv2v3table.load(newv2v3filename)

for module in ['a','b']:
    for i in range(1,6):
        det = f'nrc{module}{i}'
        print(f'\n### Detector {det}')
        ixs_det = newv2v3table.ix_equal('detector',det)
        if i<5:
            ixs_filt = newv2v3table.ix_equal('filter',sw_reffilter,indices=ixs_det)
        else:
            ixs_filt = newv2v3table.ix_equal('filter',lw_reffilter,indices=ixs_det)
        if len(ixs_filt)!=1:
            raise RuntimeError('BUG!! Could not find filter entries!')
            
        ixs_siaf = siaftable.ix_equal('AperName',f'{det.upper()}_FULL')
        if len(ixs_siaf)!=1:
            siaftable.write(columns=['AperName','V2Ref','V3Ref','V3IdlYAngle','V3SciXAngle','V3SciYAngle'],indices=ixs_siaf)
            raise RuntimeError(f'BUG!! more than one entry for detector {det.upper()}_FULL in siaftable!')
        print('OLD values:')
        siaftable.write(columns=['AperName','V2Ref','V3Ref','V3IdlYAngle','V3SciXAngle','V3SciYAngle'],indices=ixs_siaf)
        print('NEW values:')
        newv2v3table.write(columns=['detector','filter','V2Ref_mean','V3Ref_mean','V3IdlYAngle_mean'],indices=ixs_filt)
        
        # Now calculate V3SciXAngle and V3SciYAngle
        if not (det in detector2distcoeff):
            raise RuntimeError(f'Could not find detector={det} in {detector2distcoeff}')
        distcoeff = pdastroclass()
        distcoeff.load(detector2distcoeff[det])
        AA = distcoeff.t['Sci2IdlX']
        BB = distcoeff.t['Sci2IdlY']
        betax = np.degrees(np.arctan2(-AA[1],BB[1]))
        
        V3SciYAngle = newv2v3table.t.loc[ixs_filt[0],'V3IdlYAngle_mean']
        V3SciXAngle = betax+newv2v3table.t.loc[ixs_filt[0],'V3IdlYAngle_mean']

        print(f'Calculated: new V3SciXAngle={V3SciXAngle}, V3SciYAngle={V3SciYAngle}')
        
        siaftable.t.loc[ixs_siaf[0],'V2Ref'] = newv2v3table.t.loc[ixs_filt[0],'V2Ref_mean']
        siaftable.t.loc[ixs_siaf[0],'V3Ref'] = newv2v3table.t.loc[ixs_filt[0],'V3Ref_mean']
        siaftable.t.loc[ixs_siaf[0],'V3IdlYAngle'] = newv2v3table.t.loc[ixs_filt[0],'V3IdlYAngle_mean']
        siaftable.t.loc[ixs_siaf[0],'V3SciXAngle'] = V3SciXAngle
        siaftable.t.loc[ixs_siaf[0],'V3SciYAngle'] = V3SciYAngle
        print('NEW values in SIAF Table:')
        siaftable.write(columns=['AperName','V2Ref','V3Ref','V3IdlYAngle','V3SciXAngle','V3SciYAngle'],indices=ixs_siaf)
        



### Detector nrca1
OLD values:
  AperName      V2Ref       V3Ref  V3IdlYAngle  V3SciXAngle  V3SciYAngle
NRCA1_FULL 120.620865 -527.541137    -0.546442   -90.494206    -0.546442
NEW values:
detector filter  V2Ref_mean  V3Ref_mean  V3IdlYAngle_mean
   nrca1  f210m  120.620865 -527.541137         -0.546442
Calculated: new V3SciXAngle=-90.4942061887333, V3SciYAngle=-0.54644233
NEW values in SIAF Table:
  AperName      V2Ref       V3Ref  V3IdlYAngle  V3SciXAngle  V3SciYAngle
NRCA1_FULL 120.620865 -527.541137    -0.546442   -90.494206    -0.546442

### Detector nrca2
OLD values:
  AperName      V2Ref       V3Ref  V3IdlYAngle  V3SciXAngle  V3SciYAngle
NRCA2_FULL 119.789141 -459.825901    -0.199295   -90.269161    -0.199295
NEW values:
detector filter  V2Ref_mean  V3Ref_mean  V3IdlYAngle_mean
   nrca2  f210m  119.789141 -459.825901         -0.199295
Calculated: new V3SciXAngle=-90.26916134896717, V3SciYAngle=-0.199295
NEW values in SIAF Table:
  AperName      V2Ref       V3Ref  V3IdlYAngle  V

In [25]:
# these are the columns that need to be in integer format.
for col in ['XDetSize','YDetSize','XSciSize','YSciSize','DetSciParity','DetSciYAngle','Sci2IdlDeg']:
    siaftable.t[col]=siaftable.t[col].astype(pd.Int64Dtype())

subdir = os.path.basename(PRD_dir)
print(subdir)
outrootname = f'{PRD_dir}/{subdir}.NIRCam_SIAF'
print(f'Saving {outrootname}.txt')
siaftable.write(f'{outrootname}.txt')
print(f'Saving {outrootname}.xml')
siaftable.t.to_xml(f'{outrootname}.xml',index=False)

# replace <data>,<row>, with <SiafEntries>, <SiafEntry>
# replace </data>,</row>, with </SiafEntries>, </SiafEntry>
f=open(f'{outrootname}.xml','r')
lines = f.readlines()
datastart = re.compile('\<data\>')
dataend = re.compile('\<\/data\>')
rowstart = re.compile('\<row\>')
rowend = re.compile('\<\/row\>')
for i in range(len(lines)):
    lines[i]=datastart.sub('<SiafEntries>',lines[i])
    lines[i]=dataend.sub('</SiafEntries>',lines[i])
    lines[i]=rowstart.sub('<SiafEntry>',lines[i])
    lines[i]=rowend.sub('</SiafEntry>',lines[i])
print('Testing the substitutions of <data> and <row>: the next lines should be "<SiafEntries>" and "<SiafEntry>""')
print(lines[1],lines[2])
f=open(f'{outrootname}.xml','w').writelines(lines)
print(f'Testing the xml file {outrootname}.xml...')
import pysiaf
inst_siaf = pysiaf.Siaf(filename=f'{outrootname}.xml', instrument='nircam')
print(f'Testing the xml file {outrootname}.xml successful loaded into pysiaf.Siaf()!')


v2.0_PRD_220701
Saving /Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/v2.0_PRD_220701.NIRCam_SIAF.txt
Saving /Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/v2.0_PRD_220701.NIRCam_SIAF.xml
Testing the substitutions of <data> and <row>: the next lines should be "<SiafEntries>" and "<SiafEntry>""
<SiafEntries>
   <SiafEntry>

Testing the xml file /Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/v2.0_PRD_220701.NIRCam_SIAF.xml...
Testing the xml file /Users/arest/nircam/jwst_distortions_tools/final_CRDS/v2.0_PRD_220701/v2.0_PRD_220701.NIRCam_SIAF.xml successful loaded into pysiaf.Siaf()!
